In [1]:
import os, sys
import pandas as pd

#import the functions
module_path = os.path.abspath(os.path.join('../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import rtcw_functions as RTCW

db_name = 'rtcw_db.db'
demos_folder = 'demos'

cwd = os.getcwd()
demos_path = os.path.join(cwd, '../', demos_folder)

exe_name = 'Anders.Gaming.LibTech3.exe'
demos_folder = 'demos'
root_path = 'C:\\Users\\JelleGrammens\\Documents\\GIT\\rtcw_demo_analyzer' #for executable we need a direct path

In [2]:
#get demos_dct
demos_dct = RTCW.make_dictionary(demos_path)

parsed 100 matches
parsed 200 matches
parsed 300 matches
parsed 400 matches
finished parsing all matches!


In [3]:
#load obituaries in a df
import sqlite3_connector
reload(sqlite3_connector)
db = sqlite3_connector.sqlite3_connector(os.path.join(root_path, db_name))

In [4]:
obituary_df = db.get_table_as_df(table_name= 'obituary')
player_df = db.get_table_as_df(table_name= 'player')
obituary_df = RTCW.add_match_data(obituary_df, player_df, demos_dct)
obituary_df = obituary_df.loc[obituary_df['League'] == 'bob1']

obituary table fetched in 0.38 seconds
player table fetched in 0.03 seconds


In [7]:
#example, get all 5 man panzerfausts
df_spree = RTCW.get_kill_sprees(obituary_df, 
                                maxtime_secs = 10, 
                                exclude_weapon_filter = None, 
                                minspree = 4,
                                verbose = True)

all done!


In [8]:
df_spree

,attacker,demo,end,match,md5,player,spreecount,start,weapons
0,14,02-14-2002-bob-empire-vs-da-village-214021-rou...,35485850,rtcw_2002.02.14_bob1_empire_vs_da_round3,bd5f0c1cf140bceb495413fa0d6b36f7,Da^forge,4,35485850,46-46-46-46-
1,6,BOB_Empire_vs_Krieg_02_28_2002.dm_57,41957900,rtcw_2002.02.28_bob1_empire_vs_krieg_quarter-f...,204e36f690b74c4d0c2d9ed4d6e10756,Iii-mongul,4,41950950,5-5-17-17-
2,20,BOB_Empire_vs_Krieg_02_28_2002.dm_57,43911900,rtcw_2002.02.28_bob1_empire_vs_krieg_quarter-f...,204e36f690b74c4d0c2d9ed4d6e10756,Herfk,4,43906950,46-46-46-46-
3,6,-=Ec=-Chicago.dm_57,197482000,rtcw_2002.01.29_bob1_ec_vs_ccp_round1,d89171f76fbce71d2a0e1972bee69fef,Broken-wish,4,197478350,46-46-46-17-
4,19,-=Ec=-Chicago.dm_57,198366950,rtcw_2002.01.29_bob1_ec_vs_ccp_round1,d89171f76fbce71d2a0e1972bee69fef,Shadowskill,4,198358000,17-17-17-17-
5,14,[DS-P]Pat.dm_57,18746800,rtcw_2002.01.29_bob1_dropscape_vs_ep_round1,06df8ef5d323ce9960d504c78487f38f,[ds-p]fubar,4,18738550,17-17-17-17-
6,3,[DS-P]Pat.dm_57,17745300,rtcw_2002.01.29_bob1_dropscape_vs_ep_round1,06df8ef5d323ce9960d504c78487f38f,[ep]novage,4,17737200,18-18-18-18-
7,4,[DS-P]Pat.dm_57,18549100,rtcw_2002.01.29_bob1_dropscape_vs_ep_round1,06df8ef5d323ce9960d504c78487f38f,[ds-p]gfunk,4,18539550,17-17-17-17-
8,8,S62DarkStar.dm_57,43566700,rtcw_2002.02.07_bob1_s62_vs_wildfire_round2,6a78f1e07384664fd75717c78d16562a,S62daabears,4,43566700,7-7-7-7-
9,8,S62DarkStar.dm_57,44885600,rtcw_2002.02.07_bob1_s62_vs_wildfire_round2,6a78f1e07384664fd75717c78d16562a,S62daabears,4,44885600,7-7-7-7-


In [ ]:
RTCW.cut_demos(root_path, demos_dct, df_spree, offset_start = 8, offset_end = 8, transform_to_dm_60 = True)
RTCW.generate_capture_list(df_spree, transform_to_dm_60 = True)

In [ ]:
for demo in df_spree.md5:
    for k in demos_dct:
        if demo in demos_dct[k][0]:
            print k

In [ ]:
reload(RTCW)

In [ ]:
obituary_df['bWeapon'].not